#### Class 개념 정리

###### (1) 클래스: 제품의 설계도 (신발 설계도)
###### (2) 객체: 설계도로 만든 제품 (빨간 신발, 파란 신발 등)
###### (3) 속성: 클래스안의 변수
###### (4) 메서드: 클래스 안의 함수
###### (5) 생성자: 객체를 만들 때 실행되는 함수

--------------------------

#### 1-1. MyRNNcell 클래스 구현하기 - init 함수 구현
###### [1] nn.Module을 상속하여 SimpleRNN 클래스를 정의
###### [2] SimpleRNN 클래스 생성시 입력 크기(input_size), 은닉상태크기(hidden_size), 출력벡터크기(output_size)를 매개 변수로 설정

In [37]:
import torch
import torch.nn as nn

# nn.Module은 신경망 모델의 기본 구조를 정의해주는 부모 클래스
# nn.Module(부모클래스)을 상속박아 MyRNNcell(자식클래스)를 정의
class MyRNNcell(nn.Module):
    
    # 클래스의 초기화 메서드(객체가 만들어질 때, 자동으로 호출되는 메서드)
    # input_size(입력 벡터의 크기), hidden_size(은닉 상태 벡터의 크기)
    def __init__(self, input_size, hidden_size):

        # 부모 클래스(nn.Module)에서 설정된 기능을 호출
        super(MyRNNcell, self).__init__()

        # hidden_size(은닉 상태) 크기를 설정
        self.hidden_size = hidden_size

        # 입력 가중치 초기화
        # nn.Parameter는 신경망 모델의 학습 가능한 매개변수를 정의할 때 사용하는 텐서
        # torch.randn는 주어진 크기로 텐서를 생성하며, 평균0, 표준편차1인 정규분포를 따름
        self.Wx = nn.Parameter(torch.randn(hidden_size, input_size))

        # 은닉 상태 가중치 초기화
        # 이전 은닉 상태와 곱할 가중치 설정 (hidden_size x hidden_size 크기를 가짐)
        self.Wh = nn.Parameter(torch.randn(hidden_size, hidden_size))

        # 편향 벡터 초기화
        self.b = nn.Parameter(torch.randn(hidden_size))


#### 1-2. MyRNNcell 클래스 구현하기 - forward 함수 구현
###### [1] forward 함수는 PyTorch에서 순전파를 정의하는 함수 즉 입력 데이터를 받아서 출력으로 변환하는 과정을 정의
###### [2] forward 함수는 현재 입력과 이전 은닉 상태를 받아서 새로운 은닉 상태를 계산하는 함수

In [41]:
# __init 함수 구현
class MyRNNcell(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(MyRNNcell, self).__init__()
        self.hidden_size = hidden_size
        self.input_size = input_size
        self.Wx = nn.Parameter(torch.randn(hidden_size, input_size))
        self.Wh = nn.Parameter(torch.randn(hidden_size, hidden_size))
        self.b = nn.Parameter(torch.randn(hidden_size))

    # forward 함수 구현
    # forward 함수는 x(입력 벡터)와 hidden(이전 은닉 상태 벡터)을 입력값으로 받는다
    def forward(self, x, hidden):

        # 새로운 은닉 상태 계산
        # (1) x와 Wx의 전치(행렬의 행과 열을 바꾸는 연산)를 곱하기 - 차원을 맞추기 위해 사용
        hidden = torch.tanh(torch.matmul(x, self.Wx.t()) 
                        
                        # hidden(이전은닉상태)와 Wh(은닉상태가중치)의 전치를 곱하기
                        + torch.matmul(hidden, self.Wh.t()) 

                        # b(편향을 더하기)
                        + self.b)
        return hidden

-------------------------------

#### 2. MyRNNcell 인스턴스(객체)를 통한 새로운 은닉 상태 계산
###### [1] MyRNNcell을 사용해서 새 은닉 상태 계산하기 

In [42]:
model_rnn = MyRNNcell(4,6)

 # 랜덤 시드를 설정하여 무작위 숫자가 생성될 때마다 동일한 결과를 얻도록 설정
torch.manual_seed(42)

# 랜덤 Xt(입력데이터) 생성 (input_size = 4)
Xt = torch.randn(1,4)

# ht-1(초기은닉상태) 생성 (hidden_size = 6)
ht_1 = torch.zeros(1,6)

# 모델 실행 (새로운 ht 생성 공식 적용)
ht = model_rnn(Xt, ht_1)
print("새로운 은닉 상태", ht)

새로운 은닉 상태 tensor([[-0.0175, -0.8680,  0.7162,  0.6077, -0.9997, -0.4917]],
       grad_fn=<TanhBackward0>)


---------------------------------------

#### 3. 시퀀스 데이터 처리를 위한 MyRNN 클래스 구현

#### 3-1. __init__ 함수 구현
###### MyRNN 클래스를 사용하여 RNN cell의 동작을 정의하고, 이를 MyRNN 클래스에 내장하여 전체 시퀀스를 처리하는 MyRNN 클래스를 정의

In [46]:
# MyRNN 클래스 생성 이는 nn.Module을 상속받음
class MyRNN(nn.Module):

    # 초기화 메서드를 정의
    def __init__(self, input_size, hidden_size, output_size):
        
        # 부모 클래스인 nn.Module의 초기화 메서드를 호출
        super(MyRNN, self).__init__()

        # 은닉 상태 크기 설정
        self.hidden_size = hidden_size

        # RNN의 계산 단위 담당: 입력 벡터와 이전 은닉 상태를 받아 최종 출력을 계산
        self.rnn_cell = MyRNNcell(input_size, hidden_size)

        # 출력 가중치 행렬 정의
        self.Wy = nn.Parameter(torch.randn(output_size, hidden_size))

#### 3-2. init_hidden 함수 내부 구현

In [48]:
# MyRNN 클래스: init 함수 구현
class MyRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MyRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn_cell = MyRNNcell(input_size, hidden_size)
        self.Wy = nn.Parameter(torch.randn(output_size, hidden_size)) 

################################# 위와 동일 #######################################

    # 초기 은닉상태 인스턴스 생성 - 초기값이기에 아무것도 없어 0으로 초기화
    def init_hidden(self, batch_size=1):
        return torch.zeros(batch_size, self.hidden_size)

#### 3-3. forward 함수 구현

In [47]:
# MyRNN 클래스 정의
class MyRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(MyRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn_cell = MyRNNcell(input_size, hidden_size)
        self.Wy = nn.Parameter(torch.randn(output_size, hidden_size))

    def init_hidden(self, batch_size=1):
        return torch.zeros(batch_size, self.hidden_size)

################################# 위와 동일 #######################################

    # forward 함수 정의 - x는 입력 데이터
    def forward(self, x):

        # 은닉상태의 크기 정의 - 입력데이터의 batch size와 동일
        ht = self.init_hidden(x.size(0))
        
        # 
        for i in range(x.size(1)):
            ht = self.rnn_cell(x[:, i], ht)
        
        output = torch.sigmoid(torch.matmul(ht, self.Wy.t()))  # Sigmoid activation
        return output, ht